In [1]:
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd
import numpy as np

In [43]:
propiedades = pd.read_csv('/home/mati/Desktop/properati.csv')
propiedades = propiedades.loc[propiedades.Year >= 2016,:]

In [3]:
def calcular_precio_usd(fila):
    usd,currency,precio,sup,m2 = fila
    if (not np.isnan(usd) and usd != 0):
        return usd
    if (currency == 'USD'):
        if (not np.isnan(precio) and precio != 0 ):
            return precio
    if (not np.isnan(sup) and not np.isnan(m2) and sup and m2):
        return sup * m2
    return np.nan

In [46]:
propiedades.loc[:,'price_aprox_usd'] = propiedades.loc[:,['price_aprox_usd','currency','price',\
                                                          'superficie','price_usd_per_m2']]\
         .apply(lambda x: calcular_precio_usd(x),axis = 1)

In [ ]:
propiedades = propiedades.loc[propiedades.price_aprox_usd.notnull(),:].reset_index()

In [ ]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_with_parent_names'].apply\
           (lambda x: x.split('|')[3])

#### Me quedo con las columnas superficie,place_name,property_type (y tal vez lat,lon)

In [ ]:
propiedades = propiedades.loc[:,['superficie','place_name','property_type','lat','lon','price_aprox_usd','Year','Month']]

## Encodeo los strings

In [12]:
places = list(propiedades['place_name'])
types = list(propiedades['property_type'])

In [13]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(places)
propiedades.loc[:,'place_name_encoded'] = propiedades.loc[:,'place_name'].apply(lambda x: encoder.transform([x])[0])

In [14]:
encoder = preprocessing.LabelEncoder()
encoder.fit(types)
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type'].apply(lambda x: encoder.transform([x])[0])

In [15]:
propiedades.loc[propiedades.superficie.isnull(),:] = 10

In [ ]:
propiedades.to_csv('/home/mati/Desktop/properati_knn.csv', index = False)

# KNN

In [5]:
propiedades = pd.read_csv('/home/mati/Desktop/properati_knn.csv')

In [8]:
#En principio sin lat y lon ya que no estan completos

set_entrenamiento = propiedades.loc[(propiedades.Year < 2017) | (propiedades.Month < 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd']]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd']].sample(15000)
print('entrenamiento = {}, pruebas = {}\n\n'.format(len(set_entrenamiento),len(set_pruebas)))
set_entrenamiento.info()

entrenamiento = 650820, pruebas = 15000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 650820 entries, 0 to 735421
Data columns (total 4 columns):
superficie               650820 non-null float64
place_name_encoded       650820 non-null int64
property_type_encoded    650820 non-null int64
price_aprox_usd          650820 non-null float64
dtypes: float64(2), int64(2)
memory usage: 24.8 MB


In [9]:
set_entrenamiento_datos = set_entrenamiento.loc[:,['superficie','place_name_encoded','property_type_encoded']]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [16]:
from sklearn.neighbors import KNeighborsRegressor

In [11]:
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')

In [19]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded']].\
        apply(lambda x: knn.predict(x)[0],axis = 1)

In [20]:
def error_relativo(x,medido):
    return abs(x - medido) * 100 / x

In [22]:
set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
        apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
set_pruebas.head(1)

,superficie,place_name_encoded,property_type_encoded,price_aprox_usd,resultado,error_relativo
678160,55.0,72,1,107000.0,163126.0,52.454206


In [23]:
set_pruebas.error_relativo.describe()

count    15000.000000
mean        32.970245
std        111.627276
min          0.000000
25%          7.000000
50%         16.216216
75%         31.613047
max       4829.500000
Name: error_relativo, dtype: float64

In [24]:
set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.mean()

24.113510230240433

## Pruebas con varios k y varias distancias

In [30]:
res = []

In [41]:
lista_k = [5,10]
lista_dist = [50,500]

for k in lista_k:
    for d in lista_dist:
        knn = KNeighborsRegressor(n_neighbors = k, p = d)
        knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded']].\
                apply(lambda x: knn.predict(x)[0],axis = 1)
        set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
        error = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.mean()
        cant = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.count()
        res.append((k,d,error,cant))
        print(k,' - ',d)

5  -  50
5  -  500
10  -  50
10  -  500


In [42]:
for r in res:
    print ("k = {}, dist = {}, error = {}, cantidad = {}.".format(r[0],r[1],r[2],r[3]))

k = 10, dist = 1, error = 24.07050720693464, cantidad = 14762.
k = 10, dist = 2, error = 24.113510230240433, cantidad = 14759.
k = 10, dist = 3, error = 24.117962518478, cantidad = 14759.
k = 20, dist = 1, error = 25.459824075381945, cantidad = 14716.
k = 20, dist = 2, error = 25.567620912375215, cantidad = 14711.
k = 20, dist = 3, error = 25.56908034638874, cantidad = 14711.
k = 30, dist = 1, error = 26.219269836364344, cantidad = 14699.
k = 30, dist = 2, error = 26.31727096580955, cantidad = 14696.
k = 30, dist = 3, error = 26.315937128667215, cantidad = 14694.
k = 10, dist = 4, error = 24.115759709213098, cantidad = 14759.
k = 10, dist = 5, error = 24.115759709213098, cantidad = 14759.
k = 20, dist = 4, error = 25.56836718715498, cantidad = 14711.
k = 20, dist = 5, error = 25.568382702281166, cantidad = 14711.
k = 30, dist = 4, error = 26.312515418166804, cantidad = 14694.
k = 30, dist = 5, error = 26.309756163049887, cantidad = 14694.
k = 100, dist = 1, error = 28.30099223537152, c

### -------------------------------------------------------------
## Pruebo con mas datos

In [4]:
propiedades = pd.read_csv('/home/mati/Desktop/propiedades_descripcion.csv')

/home/mati/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
propiedades = propiedades.loc[(propiedades.Year >= 2016) & (propiedades.place_name.notnull()) \
                              & (propiedades.description.notnull()),:]
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840782 entries, 1614307 to 2455308
Data columns (total 16 columns):
description                   840782 non-null object
expenses                      95850 non-null object
fecha_de_publicacion          840782 non-null object
lat                           652617 non-null float64
lon                           652617 non-null float64
place_name                    840782 non-null object
price                         752849 non-null float64
price_aprox_local_currency    752849 non-null float64
price_aprox_usd               752849 non-null float64
price_per_m2                  655753 non-null float64
price_usd_per_m2              693202 non-null float64
property_type                 840782 non-null object
state_name                    840782 non-null object
superficie                    761391 non-null float64
Year                          840782 non-null int64
distancia_villas              652617 non-null float64
dtypes: float64(9), int64(1

In [8]:
propiedades.loc[:, 'Month'] = propiedades.loc[:,'fecha_de_publicacion'].apply(lambda x: int(x.split('-')[1]))

In [9]:
propiedades.loc[:,'price_aprox_usd'] = propiedades.loc[:,['price_aprox_usd','currency','price',\
                                                          'superficie','price_usd_per_m2']]\
         .apply(lambda x: calcular_precio_usd(x),axis = 1)

In [10]:
propiedades = propiedades.loc[propiedades.price_aprox_usd.notnull(),:].reset_index()

In [11]:
propiedades = propiedades.loc[:,['superficie','place_name','property_type','lat','lon','price_aprox_usd'\
                    ,'Year','Month','description']]

### Busco datos con las descripciones

In [17]:
def tiene_cadena(cadena, lista):
    cadena = str(cadena)
    cadena = cadena.lower()
    resultado = False
    for x in lista:
        resultado = resultado or (x in cadena)
    return resultado

In [18]:
cadenas = ['vigilancia', 'seguridad', 'alarma', 'blindado', 'blindaje', 'guardia']
propiedades.loc[:,'seguridad'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [19]:
cadenas = ['gimnasio', 'gym', 'gimnacio']
propiedades.loc[:,'gimnasio'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [20]:
cadenas = ['aire frio', 'aire acondicionado', 'split', 'aires acondicionados', 'climatizador', 'central',\
          'losa radiante', 'piso radiante', 'pisoradiante', 'loza radiante', 'los radiantes', 'radante',\
          'radiador']
propiedades.loc[:,'aire'] = propiedades.loc[:, 'description'].apply(lambda x: tiene_cadena(x, cadenas))

In [21]:
propiedades.to_csv('/home/mati/Desktop/properati_descripciones_knn.csv', index = False)

# KNN

In [23]:
propiedades = pd.read_csv('/home/mati/Desktop/properati_descripciones_knn.csv')

In [24]:
#En principio sin lat y lon ya que no estan completos

set_entrenamiento = propiedades.loc[(propiedades.Year < 2017) | (propiedades.Month < 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','aire','gimnasio']]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','aire','gimnasio']].sample(15000)
print('entrenamiento = {}, pruebas = {}\n\n'.format(len(set_entrenamiento),len(set_pruebas)))
set_entrenamiento.info()

entrenamiento = 650622, pruebas = 15000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 650622 entries, 0 to 735203
Data columns (total 7 columns):
superficie               650622 non-null float64
place_name_encoded       650622 non-null int64
property_type_encoded    650622 non-null int64
price_aprox_usd          650622 non-null float64
seguridad                650622 non-null bool
aire                     650622 non-null bool
gimnasio                 650622 non-null bool
dtypes: bool(3), float64(2), int64(2)
memory usage: 26.7 MB


In [25]:
set_entrenamiento_datos = set_entrenamiento.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                                                   'seguridad','aire','gimnasio']]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [69]:
res = []

In [ ]:
lista_k = [5,10,15,20,30,40,50]
lista_dist = [1,2,3]

for k in lista_k:
    for d in lista_dist:
        knn = KNeighborsRegressor(n_neighbors = k, p = d)
        knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded']].\
                apply(lambda x: knn.predict(x)[0],axis = 1)
        set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
        error = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.mean()
        cant = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.count()
        res.append((k,d,error,cant))
        print(k,' - ',d)

In [ ]:
for r in res:
    print ("k = {}, dist = {}, error = {}, cantidad = {}.".format(r[0],r[1],r[2],r[3]))